# Let's explore the RFX MDSplus tree

## Importing and setting up stuff

In [ ]:
import MDSplus as mds
import numpy as np
import matplotlib.pyplot as plt
import sys, os, time, random
from tqdm import tqdm
import h5py as h5
print(f'Python version: {sys.version}')
print(f'MDSplus version: {mds.__version__}')

In [ ]:
#color the terminal output
def pick_random_color():
    return '\033[38;5;{}m'.format(random.randint(8, 230))
ENDC = '\033[0m'

In [ ]:
# define the shot number and tree
SHOT = 30810
rfx = mds.Tree('rfx', SHOT, 'readonly') # open the tree read-only

## Traversing the tree

In [ ]:
# traverse the tree, use MAX_DEPTH to limit the depth of the tree to traverse
# othwerwise the script will run for about 10 minutes
MAX_DEPTH = 5 # maximum depth of the tree to traverse
COLORS = [pick_random_color() for i in range(MAX_DEPTH)]

usage_depth, usage_breadth = {},{}

def traverse_tree_depth_first(max_depth, node, level=0, path='', node_type='child'):
    try: 
        if level >= max_depth: return # stop if the maximum depth is reached
        if node_type == 'child': node_name = node.node_name.upper()
        elif node_type == 'member': node_name = node.node_name.lower()
        else: raise
        path = path + '/' + COLORS[level] + node_name + ENDC # add the node name
        print(f'{path}:{node.decompile()}') 
        # get the usage/type of the node
        try: usage_depth[str(node.usage)] += 1
        except: usage_depth[str(node.usage)] = 1
        # go through the children and members of the node
        for child in node.getChildren(): # get the children of the node
            traverse_tree_depth_first(max_depth, child, level + 1, path, 'child')
        for member in node.getMembers(): # get the members of the node
            traverse_tree_depth_first(max_depth, member, level + 1, path, 'member')
    except Exception as e:
        print(path + 'ERR:' + str(e))
        pass

# do the same but without recursion
def traverse_tree_breadth_first(max_depth, head_node):
    curr_nodes = [head_node]
    for d in range(max_depth):
        print('Depth:', d)
        next_nodes = []
        for node in curr_nodes:
            try:
                preprint = COLORS[d] + "   " * d + node.node_name + ENDC
                print(f'{preprint}:{node.decompile()}') # print the node
                # get the usage/type of the node
                try: usage_breadth[str(node.usage)] += 1
                except: usage_breadth[str(node.usage)] = 1
                # get the children of the node
                for child in node.getChildren():
                    next_nodes.append(child)
                # get the members of the node
                for member in node.getMembers():
                    next_nodes.append(member)
            except: pass
        curr_nodes = next_nodes
        
head_node = rfx.getNode('\\TOP.RFX') # get the top node

# test the functions, uncomment to run
# traverse_tree_depth_first(MAX_DEPTH, head_node) # traverse the tree depth-first
# traverse_tree_breadth_first(MAX_DEPTH, head_node) # traverse the tree breadth-first

In [ ]:
print(f'Usage depth: {usage_depth}')
print(f'Usage breadth: {usage_breadth}')

previous cell full depth: 'STRUCTURE': 8776, 'SUBTREE': 78, 'DEVICE': 642, 'ACTION': 1098, 'NUMERIC': 47760, 'TEXT': 17269, 'SIGNAL': 20904, 'ANY': 29, 'AXIS': 215

In [ ]:
print(f'top nodes: {[n.node_name for n in head_node.getChildren()]}')

## Exploring Signals

In [ ]:
search_space = '\\TOP.RFX.MHD.***' # *** means all nodes at this level
# search_space = '\\TOP.RFX.EDA.***' # * means all nodes at this level
# search_space = '\\TOP.RFX.***' # whole rfx tree
signal_nodes = rfx.getNodeWild(search_space, 'Signal') # get all nodes with the name 'Signal'
print(f'Found {len(signal_nodes)} of the type Signal in the search space {search_space}')

In [ ]:
# filter out the nodes without the data
data_signals = []
for node in tqdm(signal_nodes, leave=False):
    try: data = node.data(); data_signals.append(node)
    except: pass
print(f'Found {len(data_signals)}/{len(signal_nodes)} signals with data')

In [ ]:
# keep only the signals with raw data
raw_signals = []
for node in tqdm(signal_nodes, leave=False):
    try: data = node.raw_of().data(); raw_signals.append(node)
    except: pass
print(f'Found {len(raw_signals)}/{len(data_signals)} signals with raw data')


In [ ]:
# extract data from the signals and plot them
MAX_LOAD = 0 #10 #np.inf
MAX_LOAD = min(MAX_LOAD, len(raw_signals))
# select MAX_LOAD random signals
signals = random.sample(raw_signals, MAX_LOAD)
for node in (signals):
    signal = node.data()
    times = node.dim_of().data()
    unit = node.getUnits()
    full_path = node.getFullPath()
    try: node_help = node.getHelp()
    except: node_help = ''
    if signal.shape != times.shape:
        print(f'{full_path} has mismatched signal and time shapes')
        continue
    # plot the signal
    plt.figure()
    plt.plot(times, signal)
    plt.title(f'{full_path} [{unit}]\n{node_help}')
    plt.xlabel('Time [s]')
    plt.ylabel('Signal')
    plt.show()

## Convert tree in HDF5 format

In [ ]:
def convert_tree_hdf5(head_node, new_tree_file):
    #explore all the tree and save the data to the hdf5 file
    max_depth = 7
    with h5.File(new_tree_file, 'w') as f:
        curr_nodes = [(head_node,'child')]
        tot_nodes_explored = 0
        for d in range(max_depth):
            print('Depth:', d)
            next_nodes = []
            for node, node_type in curr_nodes:
                full_path = str(node.getFullPath())[10:]#.lower() # remove the \\TOP.RFX prefix
                full_path = full_path.replace('.', '/') #convert . to /
                
                usage = str(node.usage)
                
                # check if the node has data
                try: data = node.data()
                except: data = None
                
                #check if the node has time
                try: times = node.dim_of().data()
                except: times = None
                
                # VARIOUS CHECKS
                if usage == 'SIGNAL' and data is None: # signal without data
                    print(f'{full_path} is signal but has no data'); continue
                if usage == 'SIGNAL' and data is not None and times is None: # signal with data but no time
                    print(f'{full_path} is signal with data but no time: data: {data}'); continue
                if usage == 'NUMERIC' and data is None: # numeric without data
                    print(f'{full_path} is numeric but has no data'); continue
                if data is not None and times is not None: # node with data and time
                    if usage != 'SIGNAL': # not a signal but has data and time
                        print(f'{full_path} is not signal but has data and time'); pass # should be fine
                    if data.shape != times.shape: # mismatched signal and time shapes, multidimensional signals are notable exceptions, but for now we skip them
                        print(f'{full_path} has mismatched signal and time shapes, {data.shape} != {times.shape}'); continue
                
                
                # if usage in ['SIGNAL', 'NUMERIC']: # save the data to the hdf5 file
                #     if data is not None: f.create_dataset(full_path, data=data)
                #     if times is not None: f.create_dataset(full_path + '/times', data=times)
                
                # print(f'{full_path}:{usage.lower()}')
                
                try: # get the children and members of the node
                    for child in node.getChildren(): next_nodes.append((child, 'child'))
                    for member in node.getMembers(): next_nodes.append((member, 'member'))
                except: pass # do nothing if the node has no children or members
                
            curr_nodes = next_nodes
            tot_nodes_explored += len(curr_nodes)
    
    print(f'Explored {tot_nodes_explored} nodes')

# test the function
convert_tree_hdf5(rfx.getNode('\\RFX::TOP.RFX'), 'new_tree_path.hdf5')